# Particle Tracking with MODPATH

This notebook shows how to create a particle tracking model using MODPATH.
    
## To-Do
- make the examples from a package and from a model layer faster
- update toc    
- add cross section

In [ ]:
import os
import sys

import flopy
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

import nlmod

In [ ]:
nlmod.util.get_color_logger("INFO")
nlmod.show_versions()

## Groundwater Flow Model

We use the groundwater flow model from the [03_local_grid_refinement notebook](03_local_grid_refinement.ipynb). Make sure to run that notebook before you run this notebook.

In [ ]:
# load lgr model dataset
model_ws = os.path.join("..", "examples", "ijmuiden")
model_name = "IJm_planeten"

ds = xr.open_dataset(os.path.join(model_ws, f"{model_name}.nc"))

In [ ]:
# load simulation and groundwaterflow model
# set exe_name to point to mf6 version in nlmod bin directory
exe_name = os.path.join(os.path.dirname(nlmod.__file__), "bin", "mf6")
if sys.platform.startswith("win"):
    exe_name += ".exe"

sim = flopy.mf6.MFSimulation.load("mfsim.nam", sim_ws=model_ws, exe_name=exe_name)
gwf = sim.get_model(model_name=model_name)

## Modpath

### Backward tracking

In [ ]:
# list with xy coordinates to start particle tracking from
xy_start = [(101500, 496500), (101500, 499100)]

# create a modpath model
mpf = nlmod.modpath.mpf(gwf)

# create the basic modpath package
_mpfbas = nlmod.modpath.bas(mpf)

# find the nodes for given xy
nodes = nlmod.modpath.xy_to_nodes(xy_start, mpf, ds, layer=5)

# create a particle tracking group at the cell faces
pg = nlmod.modpath.pg_from_fdt(nodes)

# create the modpath simulation file
mpsim = nlmod.modpath.sim(mpf, pg, "backward", gwf=gwf)

In [ ]:
# run modpath model
nlmod.modpath.write_and_run(mpf, script_path="10_modpath.ipynb")

In [ ]:
pdata = nlmod.modpath.load_pathline_data(mpf)

In [ ]:
f, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 10))
ax.set_aspect("equal")
ax = nlmod.plot.modelgrid(ds, ax=ax)

for pid in np.unique(pdata["particleid"]):
    pf = pdata[pdata["particleid"] == pid]
    ax.plot(pf["x"], pf["y"], color="k", linewidth=0.5)
ax.plot(pf["x"], pf["y"], color="k", linewidth=0.5, label="pathline")

cids = [nlmod.grid.get_icell2d_from_xy(xy[0], xy[1], ds) for xy in xy_start]
ax.plot(
    ds.x[cids],
    ds.y[cids],
    label="start of backwards tracking",
    ls="",
    marker="o",
    color="red",
)
ax.set_title("pathlines")
ax.legend(loc="upper right");

In [ ]:
f, ax = plt.subplots(nrows=1, ncols=1, figsize=(12, 4))

for _, pid in enumerate(np.unique(pdata["particleid"])):
    pf = pdata[pdata["particleid"] == pid]
    x0, y0, z0 = pf[["x", "y", "z"]][0]
    distance = np.sqrt((pf["x"] - x0) ** 2 + (pf["y"] - y0) ** 2 + (pf["z"] - z0) ** 2)
    ax.plot(pf["time"] / 365.25, distance, label=pid)

ax.set_ylabel("distance [m]")
ax.set_xlabel("time [year]")
ax.set_title("distance travelled per particle")
ax.grid()

### Forward tracking

In [ ]:
# list with xy coordinates to start particle tracking from
xy_start = [(101500, 496500), (101500, 499100)]

# create a modpath model
mpf = nlmod.modpath.mpf(gwf)

# create the basic modpath package
_mpfbas = nlmod.modpath.bas(mpf)

# find the nodes for given xy
nodes = nlmod.modpath.xy_to_nodes(xy_start, mpf, ds, layer=5)

# create a particle tracking group at the cell faces
# pg = nlmod.modpath.pg_from_pd(nodes, localx=0.5, localy=0.5, localz=1.0)
pg = nlmod.modpath.pg_from_fdt(nodes)

# create the modpath simulation file
mpsim = nlmod.modpath.sim(mpf, pg, "forward")

In [ ]:
# run modpath model
nlmod.modpath.write_and_run(mpf, script_path="10_modpath.ipynb")

In [ ]:
pdata = nlmod.modpath.load_pathline_data(mpf)

In [ ]:
f, axl = plt.subplots(nrows=1, ncols=3, figsize=(30, 10))
for i, ax in enumerate(axl):
    ax.set_aspect("equal")
    ax = nlmod.plot.modelgrid(ds, ax=ax)

    for pid in np.unique(pdata["particleid"]):
        pf = pdata[pdata["particleid"] == pid]
        ax.plot(pf["x"], pf["y"], color="k", linewidth=0.5)
    ax.plot(pf["x"], pf["y"], color="k", linewidth=0.5, label="pathline")

    cids = [nlmod.grid.get_icell2d_from_xy(xy[0], xy[1], ds) for xy in xy_start]
    ax.plot(
        ds.x[cids],
        ds.y[cids],
        label="start of forward tracking",
        ls="",
        marker="o",
        color="red",
    )
    ax.set_title("pathlines")
    ax.legend(loc="upper right")

    if i == 1:
        ax.set_xlim(101200, 101700)
        ax.set_ylim(498700, 499300)
    elif i == 2:
        ax.set_xlim(101200, 101700)
        ax.set_ylim(496300, 496700)

In [ ]:
f, ax = plt.subplots(nrows=1, ncols=1, figsize=(12, 4))

for _, pid in enumerate(np.unique(pdata["particleid"])):
    pf = pdata[pdata["particleid"] == pid]
    x0, y0, z0 = pf[["x", "y", "z"]][0]
    distance = np.sqrt((pf["x"] - x0) ** 2 + (pf["y"] - y0) ** 2 + (pf["z"] - z0) ** 2)
    ax.plot(pf["time"] / 365.25, distance, label=pid)

ax.set_ylabel("distance [m]")
ax.set_xlabel("time [year]")
ax.set_title("distance travelled per particle")
ax.grid()

### Backward tracking from general head boundaries

In [ ]:
# create a modpath model
mpf = nlmod.modpath.mpf(gwf)

# create the basic modpath package
_mpfbas = nlmod.modpath.bas(mpf)

# get the nodes from a package
nodes = nlmod.modpath.package_to_nodes(gwf, "GHB", mpf)

# create a particle tracking group from cell centers
pg = nlmod.modpath.pg_from_pd(nodes, localx=0.5, localy=0.5, localz=0.5)

# create the modpath simulation file
mpsim = nlmod.modpath.sim(mpf, pg, "backward", gwf=gwf)

In [ ]:
# run modpath model
nlmod.modpath.write_and_run(mpf, script_path="10_modpath.ipynb")

In [ ]:
pdata = nlmod.modpath.load_pathline_data(mpf)

In [ ]:
f, axl = plt.subplots(nrows=1, ncols=3, figsize=(30, 10))
for i, ax in enumerate(axl):
    ax.set_aspect("equal")
    ax = nlmod.plot.modelgrid(ds, ax=ax)

    for pid in np.unique(pdata["particleid"]):
        pf = pdata[pdata["particleid"] == pid]
        ax.plot(pf["x"], pf["y"], color="k", linewidth=0.5)
    ax.plot(pf["x"], pf["y"], color="k", linewidth=0.5, label="pathline")

    if i > 0:
        cids = np.where((ds["rws_oppwater_cond"] != 0).values)[0]
        ax.plot(
            ds.x[cids],
            ds.y[cids],
            label="start of backwards tracking",
            ls="",
            marker="o",
            color="red",
        )
    ax.set_title("pathlines")
    ax.legend(loc="upper right")

    if i == 1:
        ax.set_xlim(101000, 102000)
        ax.set_ylim(498300, 499300)
    elif i == 2:
        ax.set_xlim(101000, 102000)
        ax.set_ylim(496300, 497300)

In [ ]:
f, ax = plt.subplots(nrows=1, ncols=1, figsize=(12, 4))

for i, pid in enumerate(np.unique(pdata["particleid"])):
    pf = pdata[pdata["particleid"] == pid]
    x0, y0, z0 = pf[["x", "y", "z"]][0]
    distance = np.sqrt((pf["x"] - x0) ** 2 + (pf["y"] - y0) ** 2 + (pf["z"] - z0) ** 2)
    ax.plot(pf["time"] / 365.25, distance, label=pid)

ax.set_xlim(0, 5000)
ax.set_ylabel("distance [m]")
ax.set_xlabel("time [year]")
ax.set_title("distance travelled per particle")
ax.grid()

### Forward tracking from each cell in the top layer

Stop after 10 years.

In [ ]:
# create a modpath model
mpf = nlmod.modpath.mpf(gwf)

# create the basic modpath package
_mpfbas = nlmod.modpath.bas(mpf)

# get nodes of all cells in the top modellayer
nodes = nlmod.modpath.layer_to_nodes(mpf, 0)

# create a particle tracking group from cell centers
pg = nlmod.modpath.pg_from_pd(nodes, localx=0.5, localy=0.5, localz=0.5)

# create the modpath simulation file
mpsim = nlmod.modpath.sim(mpf, pg, "forward", gwf=gwf, stoptime=10 * 365)

In [ ]:
# run modpath model
nlmod.modpath.write_and_run(mpf, script_path="10_modpath.ipynb")

In [ ]:
pdata = nlmod.modpath.load_pathline_data(mpf)

In [ ]:
f, axl = plt.subplots(nrows=1, ncols=3, figsize=(30, 10))
for i, ax in enumerate(axl):
    ax.set_aspect("equal")
    ax = nlmod.plot.modelgrid(ds, ax=ax)

    for pid in np.unique(pdata["particleid"]):
        pf = pdata[pdata["particleid"] == pid]
        ax.plot(pf["x"], pf["y"], color="k", linewidth=0.5)
    ax.plot(pf["x"], pf["y"], color="k", linewidth=0.5, label="pathline")

    if i > 0:
        ax.plot(
            ds.x.values,
            ds.y.values,
            label="start of forward tracking",
            ls="",
            marker="o",
            color="red",
        )
    ax.set_title("pathlines")
    ax.legend(loc="upper right")

    if i == 1:
        ax.set_xlim(101000, 102000)
        ax.set_ylim(498300, 499300)
    elif i == 2:
        ax.set_xlim(101000, 102000)
        ax.set_ylim(496300, 497300)

In [ ]:
f, ax = plt.subplots(nrows=1, ncols=1, figsize=(12, 4))

for i, pid in enumerate(np.unique(pdata["particleid"])):
    pf = pdata[pdata["particleid"] == pid]
    x0, y0, z0 = pf[["x", "y", "z"]][0]
    distance = np.sqrt((pf["x"] - x0) ** 2 + (pf["y"] - y0) ** 2 + (pf["z"] - z0) ** 2)
    ax.plot(pf["time"] / 365.25, distance, label=pid)

ax.set_xlim(0, 11)
ax.set_ylabel("distance [m]")
ax.set_xlabel("time [year]")
ax.set_title("distance travelled per particle")
ax.grid()